## Import libraries

In [1]:
from music21 import *
import glob
from tqdm import tqdm
import numpy as np
import random
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.models import Sequential, Model, load_model
from sklearn.model_selection import train_test_split

## Reading ans Parsing the Midi File

In [2]:
def read_files(file):
    notes=[]
    notes_to_parse=None
     #parse the midi file
    midi=converter.parse(file)
     #seperate all instruments from the file
    instrmt=instrument.partitionByInstrument(midi)

    for part in instrmt.parts:
     #fetch data only of Piano instrument
         if 'Piano' in str(part):
             notes_to_parse=part.recurse()

     #iterate over all the parts of sub stream elements
     #check if element's type is Note or chord
     #if it is chord split them into notes
    for element in notes_to_parse:
        if type(element)==note.Note:
              notes.append(str(element.pitch))
        elif type(element)==chord.Chord:
              notes.append('.'.join(str(n) for n in element.normalOrder))

#return the list of notes
    return notes

#retrieve paths recursively from inside the directories/files
file_path=['schubert']
print(file_path[0]+'/*.mid')
all_files=glob.glob(file_path[0]+'/*.mid',recursive=True)


#reading each midi file
notes_array = np.array([read_files(i) for i in tqdm(all_files,position=0,leave=True)])

schubert/*.mid


  0%|                                                                                           | 0/29 [00:00<?, ?it/s]c:\users\h p\appdata\local\programs\python\python39\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2009 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
  3%|██▊                                                                                | 1/29 [00:52<24:29, 52.48s/it]c:\users\h p\appdata\local\programs\python\python39\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2010 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
 14%|███████████▍                                                                       | 4/29 [02:07<12:01, 28.88s/it]c:\users\h p\ap

## Exploring the dataset

In [3]:
#unique notes
notess= sum(notes_array, [])
unique_notes= list(set(notess))
print("Unique Notes:", len(unique_notes))

#notes with their frequency
freq= dict(map(lambda x: (x, notess.count(x)), unique_notes))

#get the threshold frequency
for i in range (30, 100, 20):
    print(i,":", len(list(filter(lambda x:x[1]>=i, freq.items()))))

Unique Notes: 95
30 : 17
50 : 10
70 : 5
90 : 3


In [4]:
#filter notes greater than threshold i.e. 50
freq_notes= dict(filter(lambda x:x[1]>=50, freq.items()))

#create new notes using the frequent notes
new_notes=[[i for i in j if i in freq_notes] for j in notes_array]

In [5]:
#dictionary having key as note index ans value as note
ind2note = dict(enumerate(freq_notes))

#dictionary having key as note ans value as note index
note2ind= dict(map(reversed, ind2note.items()))

## Input and Output Sequence for model

In [6]:
#timestep
timesteps= 50

#store values of input and output
x=[]; y=[]

for i in new_notes:
    for j in range(0, len(i)-timesteps):
        #input will be the current index + timestep
        #output will be the next index after timestep
        inp= i[j:j+timesteps]; out=i[j+timesteps]
        
        #append the index value of respective notes
        x.append(list(map(lambda x:note2ind[x], inp)))
        y.append(note2ind[out])

x_new= np.array(x);
y_new= np.array(y)
    

## Training and Testing sets

In [7]:
#reshape input ans output for the model
x_new= np.reshape(x_new, (len(x_new), timesteps, 1))
y_new= np.reshape(y_new, (-1, 1))

#split the input ans value into training and testing sets
#80% for training ans 20% for testing sets

x_train,x_test,y_train,y_test= train_test_split(x_new,y_new, test_size=0.2, random_state=42)

## Building the model

In [8]:
#create the model
model= Sequential()

#create two stacked LSTM layer with the latent dimension of 256
model.add(LSTM(256, return_sequences=True, input_shape=(x_new.shape[1], x_new.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))

#fully connected layer for the output with softmax activation
model.add(Dense(len(note2ind), activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 256)           264192    
                                                                 
 dropout (Dropout)           (None, 50, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 857,866
Trainable params: 857,866
Non-trai

## Train the model

In [9]:
#compile the model using Adam optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#train the model on training sets ans validate on testing sets
model.fit(x_train, y_train, batch_size=128, epochs=80, validation_data=(x_test, y_test))

Epoch 1/80
5/5 [==============================] - 13s 1s/step - loss: 2.2621 - accuracy: 0.1693 - val_loss: 2.1786 - val_accuracy: 0.1975
Epoch 2/80
5/5 [==============================] - 4s 801ms/step - loss: 2.1529 - accuracy: 0.2348 - val_loss: 1.9771 - val_accuracy: 0.3312
Epoch 3/80
5/5 [==============================] - 4s 805ms/step - loss: 2.0031 - accuracy: 0.2796 - val_loss: 1.9012 - val_accuracy: 0.3885
Epoch 4/80
5/5 [==============================] - 4s 793ms/step - loss: 1.8849 - accuracy: 0.3482 - val_loss: 1.8268 - val_accuracy: 0.3439
Epoch 5/80
5/5 [==============================] - 4s 799ms/step - loss: 1.8248 - accuracy: 0.3243 - val_loss: 1.7546 - val_accuracy: 0.3503
Epoch 6/80
5/5 [==============================] - 4s 840ms/step - loss: 1.7409 - accuracy: 0.3275 - val_loss: 1.6437 - val_accuracy: 0.3312
Epoch 7/80
5/5 [==============================] - 4s 812ms/step - loss: 1.6655 - accuracy: 0.3770 - val_loss: 1.5701 - val_accuracy: 0.4140
Epoch 8/80
5/5 [======

5/5 [==============================] - 4s 836ms/step - loss: 0.8853 - accuracy: 0.6070 - val_loss: 1.2598 - val_accuracy: 0.5096
Epoch 60/80
5/5 [==============================] - 4s 843ms/step - loss: 0.8637 - accuracy: 0.6310 - val_loss: 1.3254 - val_accuracy: 0.4841
Epoch 61/80
5/5 [==============================] - 4s 814ms/step - loss: 0.8448 - accuracy: 0.6326 - val_loss: 1.2243 - val_accuracy: 0.4713
Epoch 62/80
5/5 [==============================] - 4s 876ms/step - loss: 0.8461 - accuracy: 0.6502 - val_loss: 1.2687 - val_accuracy: 0.4968
Epoch 63/80
5/5 [==============================] - 4s 846ms/step - loss: 0.8277 - accuracy: 0.6534 - val_loss: 1.2271 - val_accuracy: 0.5159
Epoch 64/80
5/5 [==============================] - 4s 812ms/step - loss: 0.8121 - accuracy: 0.6406 - val_loss: 1.2761 - val_accuracy: 0.4650
Epoch 65/80
5/5 [==============================] - 4s 823ms/step - loss: 0.7790 - accuracy: 0.6629 - val_loss: 1.2410 - val_accuracy: 0.5605
Epoch 66/80
5/5 [========

## Save model

In [10]:
import pickle

In [11]:
pickle.dump(model, open('model.pkl', 'wb'))

INFO:tensorflow:Assets written to: ram://eefe88cc-ed9a-4d1a-9243-7160e10ce1d1/assets


INFO:tensorflow:Assets written to: ram://eefe88cc-ed9a-4d1a-9243-7160e10ce1d1/assets
